In [8]:
# Yasir Hassan
# Writing Assignment
# KNN with Parameter Optimization
# To classify the 5 classes of SingleCell-Seq 
# using the database Data_Glioblastoma5Patients_SC

In [9]:
# import libraries
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import cv2
import scipy
import numpy as np
import pandas as pd
import os
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV 

In [10]:
# read the dataset\n",
readdata = pd.read_csv(r"C:\Users\NeverGiveUp\Desktop\Writing_File\Data_Glioblastoma5Patients_SC.csv")
df = pd.DataFrame(readdata)   

In [11]:
correct_Pattern_labels = df['Classes'].values

In [12]:
df.describe()

,A2M,AAAS,AAK1,AAMP,AARS,AARSD1,AASDH,AASDHPPT,AASS,AATF,...,ZSWIM6,ZSWIM7,ZUFSP,ZW10,ZWILCH,ZXDC,ZYG11B,ZYX,ZZZ3,Classes
count,4.300000e+02,4.300000e+02,4.300000e+02,4.300000e+02,4.300000e+02,4.300000e+02,4.300000e+02,4.300000e+02,4.300000e+02,4.300000e+02,...,4.300000e+02,4.300000e+02,4.300000e+02,4.300000e+02,4.300000e+02,4.300000e+02,4.300000e+02,4.300000e+02,4.300000e+02,430.000000
mean,-5.116304e-11,9.069751e-11,1.627911e-11,-7.674423e-11,-8.139528e-11,-1.302326e-10,-2.255814e-10,-2.325854e-12,5.813938e-11,-1.511630e-10,...,-1.906975e-10,-3.953480e-11,-4.418617e-11,-1.651161e-10,1.906977e-10,-1.348837e-10,3.255816e-11,-2.023256e-10,-1.604652e-10,2.727907
std,4.907659e+00,3.641663e+00,2.474651e+00,3.563029e+00,3.909578e+00,3.317166e+00,2.883981e+00,3.759008e+00,3.236368e+00,3.461157e+00,...,2.730348e+00,2.946168e+00,2.702606e+00,2.836726e+00,2.788284e+00,2.859633e+00,2.811249e+00,3.001418e+00,3.052755e+00,1.438049
min,-3.801470e+00,-3.889900e+00,-3.985616e+00,-5.820241e+00,-6.041792e+00,-3.699171e+00,-2.473517e+00,-4.354127e+00,-6.493931e+00,-2.570976e+00,...,-1.821098e+00,-3.118284e+00,-1.538324e+00,-1.550699e+00,-1.558581e+00,-3.068620e+00,-3.421540e+00,-2.509017e+00,-2.149696e+00,1.000000
25%,-3.801470e+00,-3.889900e+00,-1.871573e+00,-3.807913e+00,-4.560471e+00,-3.699171e+00,-2.473517e+00,-4.354127e+00,-1.967954e+00,-2.570976e+00,...,-1.821098e+00,-3.118284e+00,-1.538324e+00,-1.550699e+00,-1.558581e+00,-3.068620e+00,-2.423300e+00,-2.509017e+00,-2.149696e+00,1.000000
50%,-3.801470e+00,4.099148e-01,-3.150282e-01,1.746972e+00,1.776033e+00,1.097698e-01,-2.473517e+00,1.255356e+00,1.192040e+00,-2.570976e+00,...,-1.821098e+00,-8.136609e-01,-1.538324e+00,-1.550699e+00,-1.558581e+00,-8.339072e-01,-4.421617e-01,-2.509017e+00,-2.149696e+00,3.000000
75%,4.943281e+00,3.711243e+00,2.002047e+00,2.686033e+00,3.116038e+00,3.238343e+00,2.622538e+00,3.553135e+00,2.261182e+00,3.911515e+00,...,2.038805e+00,2.848862e+00,8.320662e-01,-1.687454e-01,3.335339e-01,2.459752e+00,2.514619e+00,2.939355e+00,2.228821e+00,4.000000
max,1.170195e+01,5.840230e+00,5.389576e+00,4.520579e+00,6.366184e+00,6.012985e+00,6.798128e+00,5.729251e+00,4.324112e+00,6.992394e+00,...,7.931653e+00,6.105923e+00,8.186486e+00,8.073628e+00,7.998381e+00,5.950488e+00,6.703674e+00,6.918941e+00,8.064823e+00,5.000000


In [13]:
# Map the labels to the ordinal numbers
df['Classes']=df['Classes'].replace({1:"CT1", 2:"CT2", 3:"CT3", 4:"CT4", 5:"CT5"})

In [14]:
# Obtain the class distribution
df['Classes'].value_counts()

CT1    118
CT2     94
CT3     75
CT4     73
CT5     70
Name: Classes, dtype: int64

In [15]:
# Train, test split\n
X = df.drop(['Classes'],axis=1)
y = df['Classes']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)

In [16]:
# obtain class labels for future use
classes = y.value_counts().index.tolist()

In [17]:
# instantiate the object for the classifier
knn = KNeighborsClassifier(n_neighbors = 7)

In [18]:
# fit the classifier with training data
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=7, p=2,
                     weights='uniform')

In [19]:
# perfrom prediction on X test data
y_pred = knn.predict(X_test)

In [20]:
print(y_pred)

['CT4' 'CT4' 'CT3' 'CT3' 'CT5' 'CT3' 'CT3' 'CT1' 'CT3' 'CT3' 'CT5' 'CT2'
 'CT3' 'CT3' 'CT3' 'CT1' 'CT1' 'CT1' 'CT3' 'CT5' 'CT2' 'CT3' 'CT1' 'CT5'
 'CT1' 'CT2' 'CT3' 'CT3' 'CT2' 'CT3' 'CT2' 'CT5' 'CT3' 'CT3' 'CT1' 'CT3'
 'CT2' 'CT2' 'CT1' 'CT3' 'CT1' 'CT1' 'CT1' 'CT2' 'CT2' 'CT4' 'CT4' 'CT3'
 'CT3' 'CT3' 'CT3' 'CT1' 'CT3' 'CT3' 'CT3' 'CT5' 'CT3' 'CT3' 'CT3' 'CT4'
 'CT3' 'CT1' 'CT2' 'CT3' 'CT3' 'CT3' 'CT3' 'CT5' 'CT3' 'CT5' 'CT3' 'CT4'
 'CT3' 'CT3' 'CT3' 'CT1' 'CT1' 'CT3' 'CT3' 'CT3' 'CT1' 'CT3' 'CT3' 'CT1'
 'CT3' 'CT1']


In [21]:
# get the confusion accuracy by comparing the prediction result to the test labels, y_test
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[18  1  4  0  0]
 [ 0  8 11  0  0]
 [ 0  0 15  0  0]
 [ 0  1  7  6  1]
 [ 0  0  7  0  7]]


In [22]:
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.627906976744186


In [23]:
# Elect  parameters for the grid_search to find the best params and the best model.
# parameters here are just estimation
parameters = [{'n_neighbors':[3]}]

In [24]:
# use GridSearchCV for parameter optimzation
grid_search = GridSearchCV(estimator = knn, param_grid = parameters,
                          scoring = 'accuracy', cv = 5, n_jobs = -1)

In [25]:
# fit the grid_search with the training data
grid_search = grid_search.fit(X_train, y_train)

In [26]:
# get the accuracy
accuracy = grid_search.best_score_
print(accuracy)

0.6831628303495311


In [27]:
# get best parameter from the grid_search
print(grid_search.best_params_)

{'n_neighbors': 3}


In [28]:
knn = KNeighborsClassifier(n_neighbors = 3)

In [29]:
# fit the model again for param. optimization
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [30]:
y_pred = knn.predict(X_test)

In [31]:
# print the confusion matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[16  1  6  0  0]
 [ 0  8 11  0  0]
 [ 0  0 15  0  0]
 [ 1  1  6  7  0]
 [ 0  0  7  0  7]]


In [32]:
# print the accuracy after parameter optimization
accuracy

0.6831628303495311